# From OMOP to IDEA4RC Episode Event

In [ ]:
import mysql.connector
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

curOMOP = conn.cursor()

In [ ]:
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

# Doubts

### OMOP to Episode Event

What to do with Disease status because the db data does not match the tada in the excel.
32946, 32947, 32949,32948=>'FIRST_EPISODE','PROGRESSION','RECURRENCE','STABLE_DISEASE'


In [ ]:
queryGetEpisode="""SELECT id FROM cancer_episode
    WHERE patient_id=%s AND date_of_diagnosis=%s;"""

selectAllQuery="""SELECT person_id, episode_start_date, episode_parent_id, episode_id FROM episode 
    WHERE episode_parent_id IS NOT NULL AND episode_concept_id IN (32946, 32947, 32949,32948);
"""  

queryGetDataParentEpisode="""SELECT episode_start_date FROM episode WHERE episode_id=%s;"""

insertQuery="""
    INSERT INTO cancer_episode (date_of_episode, defined_at, event_type, cancer_episode) VALUES (%s, %s, %s, %s);"""
queryDefinedAt="""
    SELECT care_site_id FROM provider WHERE provider_id IN(SELECT provider_id FROM procedure_occurrence
    INNER JOIN episode_event ON procedure_occurence.procedure_id=episode_event.event_id WHERE event_id=%s AND episode_event_field_concept_id=1147129);"""

curOMOP.execute(selectAllQuery)
episodes = curOMOP.fetchall()
for episode in episodes:
    curOMOP.execute(queryGetDataParentEpisode, (episode[2]))
    parentEpisodeDate = curOMOP.fetchone()
    curIDEA.execute(queryGetEpisode, (episode[0], parentEpisodeDate[0]))
    episodeId = curIDEA.fetchone()
    curOMOP.execute(queryDefinedAt,(episode[3]))
    care_site=curOMOP.fetchone()
    curIDEA.execute(insertQuery, (episode[1], care_site, ???, episodeId[0]))


conn2.commit()


### Is diagnosis...?

We need the measurement concept id to be able to do this

In [ ]:
getMeasurementsIsWrongIsNew="""
    SELECT 
"""